# Imports

In [1]:

import pandas as pd
import numpy as np
from datetime import datetime 
import requests
import math
import re

from bs4 import BeautifulSoup

# Collect data

* coleta os 36 prodcts da vitrini

In [2]:
# Parameters
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

# url do site
url = 'https://www2.hm.com/en_us/men/products/jeans.html'

# Request to URL
page = requests.get( url, headers=headers )

# Beautifullsoup
soup = BeautifulSoup( page.text, 'html.parser' )

# ===================  Product Data  =================================
products = soup.find('ul', class_= 'products-listing small')
product_list = products.find_all('article', class_ = 'hm-product-item')

# Product_id
product_list[0].get('data-articlecode')
product_id = [i.get('data-articlecode') for i in product_list]

# Product_category
product_list[0].get('data-category')
product_category = [i.get('data-category') for i in product_list]

# product_name
product_list = products.find_all('a', class_= 'item-link' )
product_list[0].get('title')
product_name = [i.get('title') for i in product_list]

# product_price
product_list = products.find_all('span', class_='price regular')
product_price = [i.get_text() for i in product_list]


data = pd.DataFrame([ product_id, product_category, product_name, product_price ]).T
data.columns = ['product_id','product_category', 'product_name', 'product_price']

# scrapy datetime
data['scrapy_datetime'] = datetime.now().strftime( '%Y-%m-%d %H:%M:%S:')
# #plotar
# data.head()

In [3]:
data.shape

(36, 5)

In [4]:
data.head()

,product_id,product_category,product_name,product_price,scrapy_datetime
0,1130309007,men_jeans_loose,Baggy Jeans,$ 39.99,2023-10-16 11:32:37:
1,1130309005,men_jeans_loose,Baggy Jeans,$ 39.99,2023-10-16 11:32:37:
2,1130309004,men_jeans_loose,Baggy Jeans,$ 39.99,2023-10-16 11:32:37:
3,1024256001,men_jeans_slim,Slim Jeans,$ 24.99,2023-10-16 11:32:37:
4,1008549001,men_jeans_regular,Straight Regular Jeans,$ 29.99,2023-10-16 11:32:37:


# Data Collection by Product

* coleta dos prod sxpecifico entra em cada um e faz as rspagem 

In [24]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime

# Dicionário que diz para o endpoint que o usuário é um Chrome (Browser)
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

df_compositions = pd.DataFrame()



# Cria uma lista vazia para adicionar as colunas que não foram coletadas anteriormente, caso um novo produto tenha alguma característica não capturada antes
aux = []

# Suponha que você tenha um DataFrame chamado "data" com informações dos produtos, como 'product_id'
for i in range(len(data)):
    URL = 'https://www2.hm.com/en_us/productpage.' + data.loc[i, 'product_id'] + '.html' 
    print('Product: {}'.format(URL))  # Correção na formatação da URL

    # Requisita essa URL do servidor do site H&M
    page = requests.get(URL, headers=headers)

    # Parâmetro de entrada para o BeautifulSoup
    soup = BeautifulSoup(page.text, 'html.parser')

    # ========================== Color Name ===============================
    
    # Product_color
    product_list = soup.find_all('a', class_='filter-option miniature')
    product_color = [i.get('data-color') for i in product_list]

    # Product_id
    product_id = [i.get('data-articlecode') for i in product_list]
    
    # Criando o DataFrame
    df_color = pd.DataFrame([ product_color,  product_id]).T
    df_color.columns = ['product_color', 'product_id']

    for j in range(len(df_color)):
        url = 'https://www2.hm.com/en_us/productpage.' + df_color.loc[j, 'product_id'] + '.html' 
        print('Color:{}'.format(url))

        page = requests.get(url, headers=headers)
        soup = BeautifulSoup(page.text, 'html.parser')

        # ===============================  =======================================
        product_elements = soup.find_all('div', class_='content pdp-text pdp-content')

        # Inicialize uma lista para armazenar as informações de todos os produtos
        all_products_info = []

        for product_element in product_elements:
            product_info = {}

            # =============================== product_name =======================================    
            product_name_element = product_element.find('h1', class_='primary product-item-headline')
            if product_name_element:
                product_name = product_name_element.text.strip()
                product_info['product_name'] = product_name
                
                
            # =============================== Product Price  =====================================
            price_tag = soup.find('span', class_='price-value')
            if price_tag:
                price = price_tag.get_text(strip=True)
                product_info['price'] = price

            # =============================== Composition =======================================
            # Extrair informações específicas para cada produto
            fit_element = product_element.find('dt', string='Fit')
            if fit_element:
                fit = fit_element.find_next('dd').text.strip()
                product_info['fit'] = fit
            
            size_element = product_element.find('dt', string='Size')
            if size_element:
                size = size_element.find_next('dd').text.strip()
                product_info['size'] = size
            
            composition_element = product_element.find('h3', string='Composition')
            if composition_element:
                composition = composition_element.find_next('ul').text.strip()
                product_info['composition'] = composition.replace('\n', '')

            article_number_element = product_element.find('div', string=re.compile(r'Article number\d+'))
            if article_number_element:
                article_number_text = article_number_element.text.strip()
                product_id = re.search(r'\d+', article_number_text).group()
                product_info['product_id'] = product_id
                product_info['product_color'] = df_color.loc[j, 'product_color']
                
            
            # Adicionar as informações do produto à lista
            all_products_info.append(product_info)

        # Criar o DataFrame
        df_composition = pd.DataFrame(all_products_info, columns=['product_name', 'price', 'fit', 'size', 'composition', 'product_id', 'product_color'])

        # Garantir o mesmo número de colunas
        df_composition = df_composition[['product_name', 'price', 'fit', 'size', 'composition', 'product_id', 'product_color']]
        
        
        # Atualizar a lista auxiliar com novas colunas
        aux = aux + df_composition.columns.tolist()

        # ========================== df_color + df_composition ==================================
        df_compositions = pd.concat([df_compositions, df_composition], axis=0)

df_compositions['style_id'] = df_compositions['product_id'].apply(lambda x: x[:-3])
df_compositions['color_id'] = df_compositions['product_id'].apply(lambda x: x[-3:])

# Definir a coluna 'scrapy_datetime' com a data e hora atual
df_compositions['scrapy_datetime'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')


Product: https://www2.hm.com/en_us/productpage.1130309007.html
Color:https://www2.hm.com/en_us/productpage.1130309001.html
Color:https://www2.hm.com/en_us/productpage.1130309002.html
Color:https://www2.hm.com/en_us/productpage.1130309004.html
Color:https://www2.hm.com/en_us/productpage.1130309005.html
Color:https://www2.hm.com/en_us/productpage.1130309008.html
Color:https://www2.hm.com/en_us/productpage.1130309009.html
Color:https://www2.hm.com/en_us/productpage.1130309011.html
Product: https://www2.hm.com/en_us/productpage.1130309005.html
Color:https://www2.hm.com/en_us/productpage.1130309001.html
Color:https://www2.hm.com/en_us/productpage.1130309002.html
Color:https://www2.hm.com/en_us/productpage.1130309004.html
Color:https://www2.hm.com/en_us/productpage.1130309007.html
Color:https://www2.hm.com/en_us/productpage.1130309008.html
Color:https://www2.hm.com/en_us/productpage.1130309009.html
Color:https://www2.hm.com/en_us/productpage.1130309011.html
Product: https://www2.hm.com/en_us

# Data Cleaning

* limpeza e separacao dados:

In [120]:
## ================================== transfommacao das col ======================================
df_data = df_compositions.dropna(subset=['product_id'])

# product_name
#df_data['product_name'] = df_data['product_name'].apply(lambda x: x.replace(' ', '_').lower())

# product_price - Remover o símbolo '$' e converter para float
df_data['price'] = df_data['price'].apply(lambda x: x.replace('$', '') if isinstance(x, str) else x ).astype(float)

## scrapy_datetime
# df_data['scrapy_datetime'] = pd.to_datetime(df_data['scrapy_datetime'], format='%Y-%m-%d %H:%M:%S:')

## Color Name
df_data['product_color'] = df_data['product_color'].apply(lambda x: x.replace(' ', '_').replace('/', '_').lower() if isinstance(x, str) else x)
#
## Fit
df_data['fit'] = df_data['fit'].apply(lambda x: x.replace(' ', '_').lower() if isinstance(x, str) else x)

## Size
df_data['size'] = df_data['size'].apply(lambda x: x.replace(' ', '_').lower() if isinstance(x, str) else x)
#
## size_number - Extraido números da coluna 'size'
df_data['size_number'] = df_data['size'].apply(lambda x: re.search('\d{3}cm', x).group(0) if isinstance(x, str) and re.search('\d{3}cm', x) else x)
df_data['size_number'] = df_data['size_number'].apply( lambda x: re.search('\d+', x).group(0) if pd.notnull( x ) else x )
#
## size_model - Extraido números da coluna 'size'
df_data['size_model'] = df_data['size'].apply( lambda x: re.search('\d+/\\d+', x).group(0) if isinstance(x, str) and re.search('\d+/\\d+', x) else x  )
#
## ========================== brack composition by comma =================================
df1 = df_data['composition'].str.split(',', expand=True)#.reset_index(drop=True) # onde tiver , quebro para outra linha

# Substitua 'Shell' e 'Pocket lining' por uma string vazia na coluna 'composition'
df1[0] = df_data['composition'].str.replace('Shell', '').str.replace('Pocket lining', '').str.replace('Lining', '').str.replace('Pocket', '')
df1[1] = df_data['composition'].str.replace('Shell', '').str.replace('Pocket lining', '').str.replace('Lining', '').str.replace('Pocket', '')

#criando um dataframe vazio do tamanho de df_data para alocar as colunas em ordem
#df_ref = pd.DataFrame( index=np.arange( len( df_data ) ), columns=['cotton', 'Polyester','Rayon','Spandex', 'Elastomultiester'] )
#
# ======= DF Cotton ====================d
#
#df_cotton = df1[0] # df_data que tem só cotton
#df_cotton.name = 'cotton' # passo o name para a coluna
###
#df_ref = pd.concat([ df_ref, df_cotton], axis=1) # faco a uniao com concat 
#df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')] # e dorp a col original q era so NaN
###
#df_ref['cotton'] = df_ref['cotton'].fillna('cotton 0%') # td coton vzio passsa a ser coton 0%
#
## ============  DF Spandex  =======================
#df_spandex = df1.loc[df1[1].str.contains('spandex', na=False), 1]
#df_spandex.name = 'spandex'
##
## combine spandex from brch colum 1 and 2
#df_spandex =df_spandex.combine_first( df1[2] )
##
#df_ref = pd.concat([df_ref, df_spandex], axis=1)
#df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')]
#df_ref['spandex'] = df_ref['spandex'].fillna('spandex 0%')
##
## ================  DF Elastomultiester  ===========================
##	
#df_Elastomultiester	 = df1.loc[ df1[1].str.contains('Elastomultiester', na=False), 1]
#df_Elastomultiester.name = 'Elastomultiester'
##
#df_ref = pd.concat([ df_ref, df_Elastomultiester], axis=1)
#df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')]
#df_ref['Elastomultiester'] = df_ref['Elastomultiester'].fillna('Elastomultiester 0%')
##
#df_ref = df_ref.drop('Spandex', axis=1)
##
## ========= Concat ================
##
##f_data = pd.concat([df_data, df_ref], axis=1)
##
##
## =====================  format composition df_data =====================================
##
## cotton
#df_data['cotton'] = df_data['cotton'].apply( lambda x: int( re.search( '\d+', x ).group(0) ) / 100 if pd.notnull( x ) else x )
##
##spandex.
#df_data['spandex'] = df_data['spandex'].apply( lambda x: int( re.search('\d+', x).group(0) ) /100 if pd.notnull(x) else x )
##
## Elastomultiester
#df_data['Elastomultiester'] = df_data['Elastomultiester'].apply( lambda x: int(re.search('\d+', x).group(0))/ 100 if pd.notnull( x ) else x)
#
#


## ============ Check =================================
#
#df_data = df_data.drop( columns=['pocket lining', 'size', 'composition'], axis=1 )
#df_data = df_data.drop_duplicates()
#df_data.shape
#
#
## criando um csv
## dados = 'data_webs_treated.csv'
## df_data.to_csv(dados, index=False)

In [121]:
df1.head()


,0,1,2,3
0,"Cotton 100%Polyester 65%, Cotton 35%","Cotton 100%Polyester 65%, Cotton 35%",None,None
0,"Cotton 100%Polyester 65%, Cotton 35%","Cotton 100%Polyester 65%, Cotton 35%",None,None
0,"Cotton 100%Polyester 65%, Cotton 35%","Cotton 100%Polyester 65%, Cotton 35%",None,None
0,"Cotton 100%Polyester 65%, Cotton 35%","Cotton 100%Polyester 65%, Cotton 35%",None,None
0,"Cotton 100%Polyester 65%, Cotton 35%","Cotton 100%Polyester 65%, Cotton 35%",None,None


In [122]:
df1[0].unique()

array(['Cotton 100%Polyester 65%, Cotton 35%',
       'Cotton 99%, Spandex 1%Polyester 65%, Cotton 35%',
       'Cotton 99%, Spandex 1%Cotton 100%',
       'Cotton 99%, Spandex 1%Polyester 63%, Cotton 37%',
       'Cotton 98%, Spandex 2%Polyester 65%, Cotton 35%', 'Cotton 100%',
       'Cotton 100%Cotton 100%', 'Cotton 98%, Spandex 2%',
       'Cotton 99%, Spandex 1%',
       'Cotton 79%, Rayon 12%, Elastomultiester 6%, Spandex 3%',
       'Cotton 98%, Spandex 2%Cotton 100%',
       'Cotton 98%, Spandex 2%Polyester 100%',
       'Cotton 99%, Spandex 1%Polyester 80%, Cotton 20%'], dtype=object)

In [123]:
df1[1].unique()

array(['Cotton 100%Polyester 65%, Cotton 35%',
       'Cotton 99%, Spandex 1%Polyester 65%, Cotton 35%',
       'Cotton 99%, Spandex 1%Cotton 100%',
       'Cotton 99%, Spandex 1%Polyester 63%, Cotton 37%',
       'Cotton 98%, Spandex 2%Polyester 65%, Cotton 35%', 'Cotton 100%',
       'Cotton 100%Cotton 100%', 'Cotton 98%, Spandex 2%',
       'Cotton 99%, Spandex 1%',
       'Cotton 79%, Rayon 12%, Elastomultiester 6%, Spandex 3%',
       'Cotton 98%, Spandex 2%Cotton 100%',
       'Cotton 98%, Spandex 2%Polyester 100%',
       'Cotton 99%, Spandex 1%Polyester 80%, Cotton 20%'], dtype=object)

In [108]:
df1[2].unique()

array([None, ' Cotton 35%', ' Cotton 37%', ' Elastomultiester 6%',
       ' Cotton 20%'], dtype=object)

In [115]:
df1[3].unique()

array([None, ' Spandex 3%'], dtype=object)